# Offense

## Runs Created (Rc)
____ 

* Estimate of a players total offensive contribution. Includes caught stealing, grounding into double plays etc that aren't included in things like BA or OPS.
* Considered the best offensive statistic you could look at.
* Total run stat, so will be lower for players that play less game

### RC27
___
* good for comparing players that may have had injuries
* RC but mulitplies the denominator by 27 to deterimine how many runs a lineup of this player would average per 9 inning game


### wRc+
____
* accounts for external facotrs like ballpark and era
* also adjusts based on avg, so a wrc+ of 110 means that that player is 10% better than the league avg

In [73]:
import pandas as pd
df = pd.read_csv("wrc_plus.csv")

In [143]:
df_wrc = df.sort_values(["Season","wRC+"], ascending=False).groupby("Season", as_index=False).head(25).sort_values(["Season", "wRC+"])

In [144]:
df_wrc["top_count"] = df_wrc.Name.map(df_wrc.Name.value_counts().to_dict())

In [147]:
df_wrc

,Season,Name,Team,wRC+,WAR,BsR,playerid,top_count
186,2015,A.J. Pollock,ARI,131,6.8,6.7,9256,1
224,2015,Ryan Braun,MIL,131,3.2,1.8,3410,2
290,2015,Brandon Belt,SFG,132,3.9,1.6,10264,2
293,2015,Lucas Duda,NYM,132,3.2,0.0,2502,1
209,2015,Michael Brantley,CLE,133,3.5,1.5,4106,1
...,...,...,...,...,...,...,...,...
71,2022,Freddie Freeman,LAD,157,7.1,5.4,5361,7
58,2022,Jose Altuve,HOU,164,6.6,-1.5,5417,5
19,2022,Paul Goldschmidt,STL,177,7.1,3.0,9218,7
16,2022,Yordan Alvarez,HOU,185,6.6,-0.6,19556,2


In [153]:
alt.Chart(df_wrc.sort_values(["Season","wRC+"], ascending=False)).mark_bar().encode(
    x="wRC+",
    y=alt.Y("Name", sort=None),
    color="top_count",
    tooltip=["Team", "WAR", "top_count", "wRC+"],
    facet=alt.Facet("Season",columns=2,
        sort=alt.EncodingSortField('Season', order='descending')  
    )
).properties(width=500, height=300)\
.resolve_scale(y="independent")

alt.Chart(...)

# Fielding

## DRS (Defensive Runs Save) and UZR (Ultimate Zone Rating)

### Explanation

* Total Defensive Runs Saved indicates how many runs a player saved or hurt his team in the field compared to the average player at his position.
* If fielder makes a play that 60% of other cf's make he gets .4 credit.

#### Notes
* data compiled by human scorers

### Measurement

* Gold Glove Caliber	+15
* Great	+10
* Above Average	+5
* Average	 0
* Below Average	-5
* Poor	-10
* Awful	-15

DRS tells you how many runs better or worse that player has been relative to the average player at his position. A +5 DRS at third means the player is five runs better than the average third baseman.


## *OOA (Outs Above Average)


* For example, if an outfielder has a ball hit to him with a 75 percent Catch Probability — that is, one an average outfielder would make three-quarters of the time — and he catches it, he’ll receive a +.25 credit. If he misses it, he’ll receive -.75, reflecting the likelihood of that ball being caught by other outfielders.

* Excellent	+10
* Above Average	1-9
* Average	 0
* Below Avg	-1 to 9
* Poor	-10

## Analysis

In [5]:
import altair as alt
import pandas as pd
import numpy as np

In [4]:
def general_scatter(df, chart_title, x, y, x_name, y_name, primary_color="white", secondary_color="black", width=1000, height=1000, circle_size=500):
    
    chart_title = chart_tile
    x_cfgs = {"shorthand":"regular_season_wins", "title":"Avg Regular Season Wins Per Season"}
    y_cfgs = {"shorthand":"pre_season_wins", "title":"Avg Preseason Wins Per Season"}
    
    # background etc.
    primary_color = primary_color
    # font etc.
    secondary_color = secondary_color


    alt.Chart(df, width=width, height=height, title=chart_title)\
    .mark_circle(size=circle_size, opacity=1)\
    .encode(
        x=alt.X(**x_cfgs,
            scale=alt.Scale(zero=False)
        ),
        y=alt.Y(**y_cfgs,
            scale=alt.Scale(zero=False)
        ),
        tooltip=[
            alt.Tooltip(**x_cfgs),
            alt.Tooltip(**y_cfgs)
        ]
    )\
    .configure(background=primary_color)\
    .configure_axis(gridColor="grey", gridDash=[6,4])\
    .configure_axisLeft(labelColor=secondary_color, titleColor=secondary_color, labelFontSize=15, titleFontSize=17)\
    .configure_axisBottom(labelColor=secondary_color, titleColor=secondary_color,labelFontSize=15, titleFontSize=17)\
    .configure_title(color=secondary_color, fontSize=25)\
    .configure_legend(
        labelColor=secondary_color, 
        titleColor=secondary_color, 
        strokeColor='gray', 
        fillColor=primary_color, 
        padding=10,
        cornerRadius=10,
        orient='top-right', 
        labelFontSize=15, 
        titleFontSize=15
    )


In [53]:
df = pd.read_csv("def.csv")

In [56]:
df_plot = df.groupby(["Name", "Team"], as_index=False).sum()
df_plot["is_cubs"] = df_plot.Team.eq("CHC")

ch = alt.Chart(df_plot, title="Scoring Error - OOA vs Errors - 2021 and 2022", width=500, height=500).mark_circle(size=100).encode(
    x=alt.X("OAA",
        scale=alt.Scale(reverse=True)
    ),
    y='E',
    color='is_cubs',
    tooltip=df_plot.columns.tolist()
)
 
ch2 = alt.Chart(df, width=500, height=500).mark_bar().encode(
    alt.Y("OAA:Q", bin=True),
    x='count()',
)

ch | ch2

MaxRowsError: The number of rows in your dataset is greater than the maximum allowed (5000). For information on how to plot larger datasets in Altair, see the documentation

alt.HConcatChart(...)

In [74]:
df = pd.read_csv("wrc_woba.csv")

In [75]:
df_plot = df[df.Season.gt(2019)].groupby(["Name", "Team", "Season"], as_index=False).sum()
df_plot["is_cubs"] = df_plot.Team.eq("CHC")

In [76]:
df_plot

,Name,Team,Season,Bat,BsR,Fld,Age,wRC+,wOBA,playerid,is_cubs
0,A.J. Alexy,TEX,2021,0.0,0.0,0.0,23,0.0,0.000,22343,False
1,A.J. Alexy,TEX,2022,0.0,0.0,0.0,24,0.0,0.000,22343,False
2,A.J. Cole,TOR,2020,0.0,0.0,0.0,28,0.0,0.000,11467,False
3,A.J. Cole,TOR,2021,0.0,0.0,0.0,29,0.0,0.000,11467,False
4,A.J. Ladwig,MIA,2022,0.0,0.0,0.0,29,0.0,0.000,16266,False
...,...,...,...,...,...,...,...,...,...,...,...
4744,Zack Thompson,STL,2022,0.0,0.0,0.0,24,0.0,0.000,25918,False
4745,Zack Weiss,LAA,2022,0.0,0.0,0.0,30,0.0,0.000,15444,False
4746,Zack Wheeler,PHI,2020,0.0,0.0,0.0,30,0.0,0.000,10310,False
4747,Zack Wheeler,PHI,2021,-10.3,0.0,0.0,31,-11.0,0.153,10310,False


In [78]:
ch = alt.Chart(df_plot, title="WRC+", width=500, height=500).mark_circle(size=100).encode(
    x=alt.X("wRC+:Q",
        scale=alt.Scale(reverse=True)
    ),
    y='wOBA',
    color='is_cubs',
    tooltip=df_plot.columns.tolist()
) 
ch

alt.Chart(...)